In [3]:
import numpy as np

from drake.Distributions import (
    sample_star_formation_rate,
    sample_fp,
    sample_ne,
    sample_fl,
    sample_fi,
    sample_fc,
    sample_L
)

from drake.Parameters import BASELINE_PARAMS


def summarize(name, x):
    print(f"\n{name}")
    print("-" * len(name))
    print(f"  n samples     : {len(x)}")
    print(f"  min           : {np.min(x):.4f}")
    print(f"  max           : {np.max(x):.4f}")
    print(f"  mean          : {np.mean(x):.4f}")
    print(f"  median        : {np.median(x):.4f}")


def main():
    n = 100_000

    print("\nRunning Drake distribution sanity checks...")
    print("=" * 45)

    R = sample_star_formation_rate(n, **BASELINE_PARAMS["R_star"])
    fp = sample_fp(n, **BASELINE_PARAMS["fp"])
    ne = sample_ne(n, **BASELINE_PARAMS["ne"])
    fl = sample_fl(n, **BASELINE_PARAMS["fl"])
    fi = sample_fi(n, **BASELINE_PARAMS["fi"])
    fc = sample_fc(n, **BASELINE_PARAMS["fc"])
    L = sample_L(n, **BASELINE_PARAMS["L"])

    summarize("R* (stars/year)", R)
    summarize("fp (planet fraction)", fp)
    summarize("ne (habitable planets)", ne)
    summarize("fl (life fraction)", fl)
    summarize("fi (intelligence fraction)", fi)
    summarize("fc (communicating fraction)", fc)
    summarize("L (civilization lifetime, years)", L)

    # Basic assertions (fail fast if something is wrong)
    assert np.all(R > 0)
    assert np.all((fp >= 0) & (fp <= 1))
    assert np.all((fl >= 0) & (fl <= 1))
    assert np.all((fi >= 0) & (fi <= 1))
    assert np.all((fc >= 0) & (fc <= 1))
    assert np.all(ne >= 0)
    assert np.all(L >= BASELINE_PARAMS["L"]["xm"])

    # Pareto tail check
    print("\nPareto tail check (L):")
    print(f"  P(L > 10,000 years): {np.mean(L > 10_000):.4f}")
    print(f"  P(L > 100,000 years): {np.mean(L > 100_000):.4f}")

    print("\n✅ All distribution checks passed!")


if __name__ == "__main__":
    main()


Running Drake distribution sanity checks...

R* (stars/year)
---------------
  n samples     : 100000
  min           : 0.5001
  max           : 2.9990
  mean          : 1.5250
  median        : 1.5145

fp (planet fraction)
--------------------
  n samples     : 100000
  min           : 0.7510
  max           : 0.9999
  mean          : 0.9615
  median        : 0.9673

ne (habitable planets)
----------------------
  n samples     : 100000
  min           : 0.0186
  max           : 1.9184
  mean          : 0.2260
  median        : 0.1993

fl (life fraction)
------------------
  n samples     : 100000
  min           : 0.0000
  max           : 1.0000
  mean          : 0.5005
  median        : 0.5006

fi (intelligence fraction)
--------------------------
  n samples     : 100000
  min           : 0.0006
  max           : 0.4788
  mean          : 0.0909
  median        : 0.0788

fc (communicating fraction)
---------------------------
  n samples     : 100000
  min           : 0.0052
  max 

In [1]:
from drake.Parameters import BASELINE_PARAMS
from drake.Simulation import simulate_once
from drake.monte_carlo import run_monte_carlo
from drake.gaussian_process import gp_sample

# Single simulation
single = simulate_once(BASELINE_PARAMS)
print("Single N:", single["N"])

# Monte Carlo
mc = run_monte_carlo(BASELINE_PARAMS, n_simulations=1_000)
print("MC Mean N:", mc["summary"]["mean"])

# GP sanity
x = list(range(20))
gp_draw = gp_sample(x)
print("GP sample OK:", len(gp_draw))

Single N: 0.013964299994069127
MC Mean N: 0.871081798285536
GP sample OK: 20


In [5]:
from drake.Copulas import gaussian_copula_sample
from drake.Sensitivity import run_prcc
from drake.utils import probability_at_least_one
from drake.monte_carlo import run_monte_carlo
from drake.Parameters import BASELINE_PARAMS

mc = run_monte_carlo(BASELINE_PARAMS, 2000)
print("P(N ≥ 1):", probability_at_least_one(mc["N"]))

prcc = run_prcc(BASELINE_PARAMS, 1000)
print(prcc)

P(N ≥ 1): 0.185
{'R_star': 0.226963654963655, 'fp': -0.029803997803997805, 'ne': 0.32390181590181594, 'fl': 0.5275056835056835, 'fi': 0.4511040191040191, 'fc': 0.3345461265461266, 'L': 0.28371977571977575}
